In [1]:
import os

In [2]:
%pwd

'd:\\DS\\Projects\\DL\\Workplace-Safety-Detection-CV\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DS\\Projects\\DL\\Workplace-Safety-Detection-CV'

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List


@dataclass(frozen=True)
class ModelPreparationAndTrainingConfig:
    root_dir: Path                      
    data_yaml_file: Path
    yolo_model: Path
    params_device: str
    params_image_size: List[int]   
    params_epochs: int                  
    params_batch_size: int              
    params_learning_rate_0: float
    params_learning_rate_f: float  
    params_augmentation: bool             
    params_augmentation_hyperparams: dict               

In [6]:
import os
from CV.constants import *
from CV.utils.common import read_yaml, create_directories
from CV.entity.config_entity import ModelPreparationAndTrainingConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_preparation_and_training_config(self) -> ModelPreparationAndTrainingConfig:
        config = self.config.model_preparation_and_training
        params = self.params

        create_directories([config.root_dir])

        model_preparation_and_training_config = ModelPreparationAndTrainingConfig(
            root_dir=Path(config.root_dir),
            data_yaml_file=Path(config.data_yaml_file),
            yolo_model=Path(config.yolo_model),
            params_device=params.DEVICE,
            params_image_size=params.IMAGE_SIZE,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_learning_rate_0=params.LEARNING_RATE_0,
            params_learning_rate_f=params.LEARNING_RATE_F,
            params_augmentation=params.AUGMENTATION,
            params_augmentation_hyperparams=params.AUGMENTATION_HYPERPARAMS,
        )

        return model_preparation_and_training_config


In [7]:
import os
from ultralytics import YOLO
from CV.entity.config_entity import ModelPreparationAndTrainingConfig
from pathlib import Path

                                                

class YOLOModelTrainer:
    def __init__(self, config: ModelPreparationAndTrainingConfig):

        self.config = config

    def prepare_and_train_yolo_model(self):
        """
        Prepare and configure the YOLO model with the specified parameters.
        Train the YOLO model using the provided data configuration.
        """
        yolo_model_path = os.path.join(self.config.root_dir, self.config.yolo_model)

        # Load YOLO model
        try:
            yolo_model = YOLO(yolo_model_path)
        except FileNotFoundError as e:
            raise FileNotFoundError(f"YOLO model file not found at {yolo_model_path}") from e

        # Apply learning rate
        yolo_model.overrides['lr0'] = self.config.params_learning_rate_0
        yolo_model.overrides['lrf'] = self.config.params_learning_rate_f

        # Conditionally apply augmentation hyperparameters
        if self.config.params_augmentation:
            yolo_model.overrides.update(self.config.params_augmentation_hyperparams)

        # Dynamically resolve the output directory
        base_dir = Path(os.getcwd()).resolve()  # Get the directory of the script
        output_dir = base_dir / self.config.root_dir

        # Ensure the output directory exists
        output_dir.mkdir(parents=True, exist_ok=True)

        # Train the YOLO model
        try:
            # Train the model
            yolo_model.train(
                data=self.config.data_yaml_file,
                epochs=self.config.params_epochs,
                imgsz=self.config.params_image_size,
                batch=self.config.params_batch_size,
                device=self.config.params_device,
                workers=0,  # Avoid multiprocessing issues
                project=str(output_dir),  # Convert Path to string for YOLOv8
                name="yolov8n_training"  # Custom name for this run
            )

            # Delete the file at the specified path if it exists.
            file = Path("yolo11n.pt")

            if file.exists():
                file.unlink()  # Delete the file
                print(f"Deleted file: {file}")
            else:
                print(f"No file found at: {file}")
        
        except Exception as e:
                raise e

In [8]:
try:
    config = ConfigurationManager()
    model_preparation_and_training_config = config.get_model_preparation_and_training_config()
    model_preparation_and_training = YOLOModelTrainer(config=model_preparation_and_training_config)
    model_preparation_and_training.prepare_and_train_yolo_model()
except Exception as e:
    raise e

[2025-01-22 21:04:46,941: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-22 21:04:46,946: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-22 21:04:46,947: INFO: common: created directory at: artifacts]
[2025-01-22 21:04:46,948: INFO: common: created directory at: artifacts/model_preparation_and_training]


100%|██████████| 6.25M/6.25M [00:00<00:00, 6.59MB/s]


Ultralytics 8.3.65  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
engine\trainer: task=detect, mode=train, model=artifacts\model_preparation_and_training\yolov8n.pt, data=artifacts\data_ingestion\Workplace-Safety-Dataset\data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=0, project=D:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\model_preparation_and_training, name=yolov8n_training, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnos

100%|██████████| 5.35M/5.35M [00:00<00:00, 6.12MB/s]


AMP: checks passed 


...

        3/3      2.36G     0.9639      1.042      1.158        139        640: 100%|██████████| 70/70 [01:20<00:00,  1.16s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:12<00:00,  1.17s/it]

                   all        325       1509      0.891      0.835      0.916      0.665



3 epochs completed in 0.076 hours.
Optimizer stripped from D:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\model_preparation_and_training\yolov8n_training\weights\last.pt, 6.2MB
Optimizer stripped from D:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\model_preparation_and_training\yolov8n_training\weights\best.pt, 6.2MB

Validating D:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\model_preparation_and_training\yolov8n_training\weights\best.pt...
Ultralytics 8.3.65  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:14<00:00,  1.36s/it]


                   all        325       1509      0.891      0.836      0.917      0.665
                 Boots        245        539      0.927      0.831      0.918       0.62
                Helmet        231        272      0.917      0.809      0.917      0.629
                Person        245        301      0.854      0.832      0.918      0.749
                  Vest        306        397      0.867      0.872      0.913       0.66
Speed: 0.5ms preprocess, 8.8ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to D:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\model_preparation_and_training\yolov8n_training
Deleted file: yolo11n.pt
